# Task 1: Strawberry and Peduncle Segmentation Training

This notebook trains a YOLO11 segmentation model to detect and segment:
- Strawberries (ripe, unripe, half-ripe)
- Peduncles (stems)

**Dataset**: Loaded from GitHub repository
**Environment**: Designed for Kaggle with GPU support

## 1. Environment Setup

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Running on CPU")

In [ ]:
# Install dependencies
!pip install -q ultralytics opencv-python-headless matplotlib seaborn scikit-learn
!pip install -q pillow numpy pandas tqdm

In [ ]:
import os
import json
import shutil
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
import cv2
from ultralytics import YOLO

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Download Dataset from GitHub

In [ ]:
# Clone dataset repository
REPO_URL = "https://github.com/SergKurchev/strawberry_synthetic_dataset.git"
DATASET_DIR = "/kaggle/working/strawberry_dataset"

if not os.path.exists(DATASET_DIR):
    print("Cloning dataset repository...")
    !git clone {REPO_URL} {DATASET_DIR}
else:
    print("Dataset already exists")

# Verify dataset structure
print("\nDataset structure:")
for item in sorted(os.listdir(DATASET_DIR)):
    path = os.path.join(DATASET_DIR, item)
    if os.path.isdir(path):
        count = len(os.listdir(path))
        print(f"  {item}/: {count} files")
    else:
        print(f"  {item}")

## 3. Analyze Dataset

In [ ]:
# Load annotations
annotations_path = os.path.join(DATASET_DIR, "annotations.json")
with open(annotations_path, 'r') as f:
    coco_data = json.load(f)

print(f"Total images: {len(coco_data['images'])}")
print(f"Total annotations: {len(coco_data['annotations'])}")
print(f"Categories: {len(coco_data['categories'])}")

# Display categories
print("\nClass mapping:")
for cat in coco_data['categories']:
    print(f"  {cat['id']}: {cat['name']}")

In [ ]:
# Analyze class distribution
from collections import Counter

class_counts = Counter([ann['category_id'] for ann in coco_data['annotations']])
class_names = {cat['id']: cat['name'] for cat in coco_data['categories']}

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Bar plot
names = [class_names[i] for i in sorted(class_counts.keys())]
counts = [class_counts[i] for i in sorted(class_counts.keys())]
colors = ['#FF6B6B', '#4ECDC4', '#FFE66D', '#95E1D3']

ax1.bar(names, counts, color=colors)
ax1.set_title('Class Distribution', fontsize=14, fontweight='bold')
ax1.set_ylabel('Count')
ax1.tick_params(axis='x', rotation=45)
for i, v in enumerate(counts):
    ax1.text(i, v + 10, str(v), ha='center', fontweight='bold')

# Pie chart
ax2.pie(counts, labels=names, autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title('Class Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('/kaggle/working/class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nClass statistics:")
for name, count in zip(names, counts):
    print(f"  {name}: {count} instances")

## 4. Prepare YOLO Format Dataset

In [ ]:
# Create YOLO dataset structure
YOLO_DIR = "/kaggle/working/yolo_dataset"
os.makedirs(f"{YOLO_DIR}/images/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/images/val", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/val", exist_ok=True)

print("YOLO dataset structure created")

In [ ]:
# Split dataset (80% train, 20% val)
from sklearn.model_selection import train_test_split

image_files = [img['file_name'] for img in coco_data['images']]
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

print(f"Train set: {len(train_files)} images")
print(f"Val set: {len(val_files)} images")

In [ ]:
# Copy images and convert labels to YOLO format
def copy_dataset(file_list, split):
    """Copy images and labels for given split"""
    file_set = set(file_list)
    
    for img_info in tqdm(coco_data['images'], desc=f"Processing {split}"):
        if img_info['file_name'] not in file_set:
            continue
            
        # Copy image
        src_img = os.path.join(DATASET_DIR, "images", img_info['file_name'])
        dst_img = os.path.join(YOLO_DIR, "images", split, img_info['file_name'])
        shutil.copy2(src_img, dst_img)
        
        # Get annotations for this image
        img_anns = [ann for ann in coco_data['annotations'] if ann['image_id'] == img_info['id']]
        
        # Convert to YOLO format
        label_file = os.path.join(YOLO_DIR, "labels", split, 
                                  img_info['file_name'].replace('.png', '.txt'))
        
        with open(label_file, 'w') as f:
            for ann in img_anns:
                # YOLO format already in labels/*.txt from Unity
                # But we'll use COCO annotations for consistency
                pass
        
        # Copy existing YOLO label
        src_label = os.path.join(DATASET_DIR, "labels", 
                                img_info['file_name'].replace('.png', '.txt'))
        if os.path.exists(src_label):
            shutil.copy2(src_label, label_file)

copy_dataset(train_files, "train")
copy_dataset(val_files, "val")

print("\nDataset preparation complete!")

In [ ]:
# Create data.yaml for YOLO
data_yaml = f"""
path: {YOLO_DIR}
train: images/train
val: images/val

names:
  0: strawberry_ripe
  1: strawberry_unripe
  2: strawberry_half_ripe
  3: peduncle

nc: 4
"""

with open(f"{YOLO_DIR}/data.yaml", 'w') as f:
    f.write(data_yaml)

print("data.yaml created")
print(data_yaml)

## 5. Visualize Training Samples

In [ ]:
# Visualize random samples with annotations
def visualize_sample(image_path, label_path, class_names):
    """Visualize image with YOLO segmentation annotations"""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    # Read labels
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            labels = f.readlines()
        
        colors = [(255, 107, 107), (78, 205, 196), (255, 230, 109), (149, 225, 211)]
        
        for label in labels:
            parts = label.strip().split()
            class_id = int(parts[0])
            points = np.array(parts[1:], dtype=float).reshape(-1, 2)
            points[:, 0] *= w
            points[:, 1] *= h
            points = points.astype(np.int32)
            
            # Draw polygon
            cv2.polylines(img, [points], True, colors[class_id], 2)
            
            # Add label
            cx, cy = points.mean(axis=0).astype(int)
            cv2.putText(img, class_names[class_id], (cx-30, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_id], 2)
    
    return img

# Show 6 random samples
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

class_names = ['strawberry_ripe', 'strawberry_unripe', 'strawberry_half_ripe', 'peduncle']
train_images = sorted(os.listdir(f"{YOLO_DIR}/images/train"))[:6]

for idx, img_name in enumerate(train_images):
    img_path = f"{YOLO_DIR}/images/train/{img_name}"
    label_path = f"{YOLO_DIR}/labels/train/{img_name.replace('.png', '.txt')}"
    
    img = visualize_sample(img_path, label_path, class_names)
    axes[idx].imshow(img)
    axes[idx].set_title(f"Sample {idx+1}", fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('/kaggle/working/training_samples.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Train YOLO11 Segmentation Model

In [ ]:
# Initialize YOLO11 model
model = YOLO('yolo11n-seg.pt')  # Use nano model for faster training
# For better accuracy, use: yolo11s-seg.pt, yolo11m-seg.pt, yolo11l-seg.pt, or yolo11x-seg.pt

print("Model loaded successfully")

In [ ]:
# Training configuration
results = model.train(
    data=f"{YOLO_DIR}/data.yaml",
    epochs=100,
    imgsz=1024,
    batch=8,  # Adjust based on GPU memory
    patience=20,
    save=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=4,
    project='/kaggle/working/runs',
    name='strawberry_segmentation',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    verbose=True,
    seed=42,
    deterministic=True,
    single_cls=False,
    rect=False,
    cos_lr=True,
    close_mosaic=10,
    amp=True,  # Automatic Mixed Precision
    fraction=1.0,
    profile=False,
    overlap_mask=True,
    mask_ratio=4,
    dropout=0.0,
    val=True,
    plots=True
)

## 7. Evaluate Model

In [ ]:
# Load best model
best_model = YOLO('/kaggle/working/runs/strawberry_segmentation/weights/best.pt')

# Validate
metrics = best_model.val(data=f"{YOLO_DIR}/data.yaml")

print("\n=== Validation Metrics ===")
print(f"Box mAP50: {metrics.box.map50:.4f}")
print(f"Box mAP50-95: {metrics.box.map:.4f}")
print(f"Mask mAP50: {metrics.seg.map50:.4f}")
print(f"Mask mAP50-95: {metrics.seg.map:.4f}")

In [ ]:
# Display training curves
from IPython.display import Image as IPImage, display

results_dir = '/kaggle/working/runs/strawberry_segmentation'

print("Training Results:")
for img_name in ['results.png', 'confusion_matrix.png', 'val_batch0_pred.png']:
    img_path = os.path.join(results_dir, img_name)
    if os.path.exists(img_path):
        print(f"\n{img_name}:")
        display(IPImage(filename=img_path))

## 8. Test Inference

In [ ]:
# Run inference on validation samples
val_images = sorted(os.listdir(f"{YOLO_DIR}/images/val"))[:6]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, img_name in enumerate(val_images):
    img_path = f"{YOLO_DIR}/images/val/{img_name}"
    
    # Predict
    results = best_model.predict(img_path, conf=0.25, iou=0.7, verbose=False)
    
    # Visualize
    result_img = results[0].plot()
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(result_img)
    axes[idx].set_title(f"Prediction {idx+1}", fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('/kaggle/working/predictions.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Export Model

In [ ]:
# Export to ONNX for deployment
best_model.export(format='onnx', dynamic=True, simplify=True)

print("Model exported to ONNX format")
print(f"\nModel files:")
!ls -lh /kaggle/working/runs/strawberry_segmentation/weights/

## 10. Save Results Summary

In [ ]:
# Create summary report
summary = {
    "model": "YOLO11n-seg",
    "dataset": {
        "total_images": len(coco_data['images']),
        "train_images": len(train_files),
        "val_images": len(val_files),
        "classes": class_names
    },
    "training": {
        "epochs": 100,
        "image_size": 1024,
        "batch_size": 8
    },
    "metrics": {
        "box_map50": float(metrics.box.map50),
        "box_map50_95": float(metrics.box.map),
        "mask_map50": float(metrics.seg.map50),
        "mask_map50_95": float(metrics.seg.map)
    }
}

with open('/kaggle/working/segmentation_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("Summary saved to segmentation_summary.json")
print(json.dumps(summary, indent=2))